<a href="https://colab.research.google.com/github/suryasphoorthi536-rgb/Automated-Underwriting-Platform/blob/main/Automated_Underwriting_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate pillow torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoFeatureExtractor, AutoModelForImageClassification, AutoModelForTokenClassification
from PIL import Image
import torch
import requests
from io import BytesIO

In [5]:
import os
from huggingface_hub import login

# Assumes you've already stored the token in your environment variables
hf_token = os.environ.get("hftoken")

# Login using the token
login(token=hf_token)

In [10]:
# Use a pretrained NER model (e.g. spaCy or HF)
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
ner = pipeline("ner", model=model, tokenizer=tokenizer)

def extract_info(text):
    ents = ner(text)
    result = {}
    for e in ents:
        result.setdefault(e['entity'], []).append(e['word'])
    return result

sample = """
123 Maple St, Springfield. Built 1995. Living area: 2,100 sqft. Issues: water damage, cracked foundation.
"""
info = extract_info(sample)
print(info)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{'B-LOC': ['Maple', 'Springfield'], 'I-LOC': ['St']}


In [12]:
from google.colab import files
from PIL import Image
import requests
from io import BytesIO
from transformers import AutoFeatureExtractor, AutoModelForImageClassification, pipeline
import torch

# Load the model and tokenizer
img_tokenizer = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
img_model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")
img_clf = pipeline("image-classification", model=img_model, feature_extractor=img_tokenizer)

# Upload file from local
uploaded = files.upload()

# Get the uploaded file name (assuming one file)
uploaded_filename = list(uploaded.keys())[0]

# Assessment function that now takes a PIL image
def assess_image_file(image_file):
    img = Image.open(image_file).convert('RGB')
    preds = img_clf(img)
    hazards = [p['label'] for p in preds if p['score'] > 0.1]
    return hazards

# Run the classification
print(assess_image_file(uploaded_filename))


/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Device set to use cpu


Saving SURYA PIC.jpg to SURYA PIC.jpg
[]


In [14]:
def compute_risk(text_info, img_hazards):
    score = 0
    score += len(text_info.get('MISC', [])) * 1.0   # e.g., every flagged entity +1
    score += len(img_hazards) * 2.0
    return min(score, 10)

risk = compute_risk(info, assess_image_file(uploaded_filename))
print(f"Risk Score: {risk}/10")

Risk Score: 0.0/10


In [15]:
GUIDELINES = {
    (0, 3): "Accept",
    (3.01, 6): "Manual Review",
    (6.01, 10): "Decline"
}

def decision(score):
    for (lo, hi), action in GUIDELINES.items():
        if lo <= score <= hi:
            return action
    return "Error"

print("Decision:", decision(risk))


Decision: Accept


In [18]:
def assess_image_url(image_url):
    try:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        preds = img_clf(img)
        hazards = [p['label'] for p in preds if p['score'] > 0.1]
        return hazards
    except Exception as e:
        print(f"Error processing image URL {image_url}: {e}")
        return []

def assess_property(report_text, image_paths_or_urls):
    info = extract_info(report_text)
    img_hazards = []
    for img_path_or_url in image_paths_or_urls:
        if img_path_or_url.startswith('http'):
            img_hazards += assess_image_url(img_path_or_url)
        else:
            img_hazards += assess_image_file(img_path_or_url)
    score = compute_risk(info, img_hazards)
    return {
        "extracted_info": info,
        "image_hazards": img_hazards,
        "risk_score": score,
        "underwriting_decision": decision(score)
    }

# Example usage
report = """
Property: 987 Oak Ave, Greenwood. Sqft: 3,500. Notes: mold on basement walls, old roof.
"""
imgs = ["SURYA PIC.jpg"]
result = assess_property(report, imgs)
print(result)

{'extracted_info': {'B-LOC': ['Oak', 'Greenwood'], 'I-LOC': ['Ave', 'S']}, 'image_hazards': [], 'risk_score': 0.0, 'underwriting_decision': 'Accept'}
